In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn.functional as F

We consider the problem of estimating the impact of assigning a different room as compared to what the customer had reserved on Booking Cancellations.

In [2]:
hotel_1 = pd.read_csv('H1.csv',parse_dates=True,index_col='ReservationStatusDate')
hotel_2 = pd.read_csv('H2.csv',parse_dates=True,index_col='ReservationStatusDate')
hotel_1.head(10)

,IsCanceled,LeadTime,ArrivalDateYear,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,StaysInWeekendNights,StaysInWeekNights,Adults,Children,...,BookingChanges,DepositType,Agent,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus
ReservationStatusDate,,,,,,,,,,,,,,,,,,,,,
2015-07-01,0,342,2015,July,27,1,0,0,2,0,...,3,No Deposit,NULL,NULL,0,Transient,0.0,0,0,Check-Out
2015-07-01,0,737,2015,July,27,1,0,0,2,0,...,4,No Deposit,NULL,NULL,0,Transient,0.0,0,0,Check-Out
2015-07-02,0,7,2015,July,27,1,0,1,1,0,...,0,No Deposit,NULL,NULL,0,Transient,75.0,0,0,Check-Out
2015-07-02,0,13,2015,July,27,1,0,1,1,0,...,0,No Deposit,304,NULL,0,Transient,75.0,0,0,Check-Out
2015-07-03,0,14,2015,July,27,1,0,2,2,0,...,0,No Deposit,240,NULL,0,Transient,98.0,0,1,Check-Out
2015-07-03,0,14,2015,July,27,1,0,2,2,0,...,0,No Deposit,240,NULL,0,Transient,98.0,0,1,Check-Out
2015-07-03,0,0,2015,July,27,1,0,2,2,0,...,0,No Deposit,NULL,NULL,0,Transient,107.0,0,0,Check-Out
2015-07-03,0,9,2015,July,27,1,0,2,2,0,...,0,No Deposit,303,NULL,0,Transient,103.0,0,1,Check-Out
2015-05-06,1,85,2015,July,27,1,0,3,2,0,...,0,No Deposit,240,NULL,0,Transient,82.0,0,1,Canceled


In [3]:
hotel_1.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 40060 entries, 2015-07-01 to 2017-09-14
Data columns (total 30 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   IsCanceled                   40060 non-null  int64  
 1   LeadTime                     40060 non-null  int64  
 2   ArrivalDateYear              40060 non-null  int64  
 3   ArrivalDateMonth             40060 non-null  object 
 4   ArrivalDateWeekNumber        40060 non-null  int64  
 5   ArrivalDateDayOfMonth        40060 non-null  int64  
 6   StaysInWeekendNights         40060 non-null  int64  
 7   StaysInWeekNights            40060 non-null  int64  
 8   Adults                       40060 non-null  int64  
 9   Children                     40060 non-null  int64  
 10  Babies                       40060 non-null  int64  
 11  Meal                         40060 non-null  object 
 12  Country                      39596 non-null  object 
 13 

This dataset contains booking information for a city hotel and a resort hotel taken from a real hotel in Portugal

## Process Null

In [4]:
print(hotel_1.isna().sum())
print(hotel_2.isna().sum())

IsCanceled                       0
LeadTime                         0
ArrivalDateYear                  0
ArrivalDateMonth                 0
ArrivalDateWeekNumber            0
ArrivalDateDayOfMonth            0
StaysInWeekendNights             0
StaysInWeekNights                0
Adults                           0
Children                         0
Babies                           0
Meal                             0
Country                        464
MarketSegment                    0
DistributionChannel              0
IsRepeatedGuest                  0
PreviousCancellations            0
PreviousBookingsNotCanceled      0
ReservedRoomType                 0
AssignedRoomType                 0
BookingChanges                   0
DepositType                      0
Agent                            0
Company                          0
DaysInWaitingList                0
CustomerType                     0
ADR                              0
RequiredCarParkingSpaces         0
TotalOfSpecialReques

In [5]:
len(hotel_1.Company.unique())

236

It can be easily seen that the null values in the Company and Agent features are not counted. Now, we need to convert them to the same way as other null value so that it will be easier to treat them later.

In [6]:
hotel_1 = hotel_1.replace(to_replace = '       NULL', 
                 value =np.NAN) 
print(hotel_1.isna().sum())
hotel_2 = hotel_2.replace(to_replace = '       NULL', 
                 value =np.NAN) 
print(hotel_2.isna().sum())

IsCanceled                         0
LeadTime                           0
ArrivalDateYear                    0
ArrivalDateMonth                   0
ArrivalDateWeekNumber              0
ArrivalDateDayOfMonth              0
StaysInWeekendNights               0
StaysInWeekNights                  0
Adults                             0
Children                           0
Babies                             0
Meal                               0
Country                          464
MarketSegment                      0
DistributionChannel                0
IsRepeatedGuest                    0
PreviousCancellations              0
PreviousBookingsNotCanceled        0
ReservedRoomType                   0
AssignedRoomType                   0
BookingChanges                     0
DepositType                        0
Agent                           8209
Company                        36952
DaysInWaitingList                  0
CustomerType                       0
ADR                                0
R

Most of the elements in feature Company is null, so it is better to delete this feature.

In [7]:
# Drop Company from both hotel_1 & hotel_2 datasets
hotel_1 = hotel_1.drop(['Company'],axis=1)
hotel_2 = hotel_2.drop(['Company'],axis=1)

# Fill NA values using Most frequently occuring value in that column
hotel_1['Country'] = hotel_1['Country'].fillna(hotel_1['Country'].mode()[0])
hotel_1['Agent'] = hotel_1['Agent'].fillna(hotel_1['Agent'].mode()[0])

hotel_2['Country'] = hotel_2['Country'].fillna(hotel_2['Country'].mode()[0])
hotel_2['Agent'] = hotel_2['Agent'].fillna(hotel_2['Agent'].mode()[0])
hotel_2['Children'] = hotel_2['Children'].fillna(hotel_2['Children'].mode()[0])

In [8]:
print(hotel_1.isna().sum())
print(hotel_2.isna().sum())

IsCanceled                     0
LeadTime                       0
ArrivalDateYear                0
ArrivalDateMonth               0
ArrivalDateWeekNumber          0
ArrivalDateDayOfMonth          0
StaysInWeekendNights           0
StaysInWeekNights              0
Adults                         0
Children                       0
Babies                         0
Meal                           0
Country                        0
MarketSegment                  0
DistributionChannel            0
IsRepeatedGuest                0
PreviousCancellations          0
PreviousBookingsNotCanceled    0
ReservedRoomType               0
AssignedRoomType               0
BookingChanges                 0
DepositType                    0
Agent                          0
DaysInWaitingList              0
CustomerType                   0
ADR                            0
RequiredCarParkingSpaces       0
TotalOfSpecialRequests         0
ReservationStatus              0
dtype: int64
IsCanceled                    

## Data Exploration

### Description of some features:

**IsCanceled:** Value indicating if the booking was canceled (1) or not (0)

**ADR:** Average Daily Rate (Calculated by dividing the sum of all lodging transactions by the total number of staying nights)

**Agent:** ID of the travel agency that made the booking

**AssignedRoomType:** Code for the type of room assigned to the booking. Sometimes the assigned room type differs from the reserved room type due to hotel operation reasons (e.g. overbooking) or by customer request

**BookingChanges:** Number of changes/amendments made to the booking from the moment the booking was entered on the PMS until the moment of check-in or cancellation

**Company:** ID of the company/entity that made the booking or responsible for paying the booking

**CustomerType:**  Type of booking, assuming one of four categories:
- Contract - when the booking has an allotment or other type of contract associated to it
- Group – when the booking is associated to a group
- Transient – when the booking is not part of a group or contract, and is not associated to other transient booking
- Transient-party – when the booking is transient, but is associated to at least other transient booking

**DaysInWaitingList:** Number of days the booking was in the waiting list before it was confirmed to the customer

**DepositType:**  Indication on if the customer made a deposit to guarantee the booking. This variable can assume three categories:
- No Deposit – no deposit was made
- Non Refund – a deposit was made in the value of the total stay cost
- Refundable – a deposit was made with a value under the total cost of stay

**DistributionChannel:** Booking distribution channel. The term “TA” means “Travel Agents” and “TO” means “Tour Operators”

**IsRepeatedGuest:** Value indicating if the booking name was from a repeated guest (1) or not (0)

**LeadTime:** Number of days that elapsed between the entering date of the booking into the PMS and the arrival date

**Meal:** Type of meal booked. Categories are presented in standard hospitality meal packages:
- Undefined/SC – no meal package
- BB – Bed & Breakfast
- HB – Half board (breakfast and one other meal – usually dinner)
- FB – Full board (breakfast, lunch and dinner)

**PreviousBookingsNotCanceled:** Number of previous bookings not cancelled by the customer prior to the current booking

**PreviousCancellations:** Number of previous bookings that were cancelled by the customer prior to the current booking

**RequiredCardParkingSpaces:** Number of car parking spaces required by the customer

**ReservationStatus:** Reservation last status, assuming one of three categories:
- Canceled – booking was canceled by the customer
- Check-Out – customer has checked in but already departed
- No-Show – customer did not check-in and did inform the hotel of the reason why

**ReservationStatusDate:** Date at which the last status was set.This variable can be used in conjunction with the ReservationStatus to understand when was the booking canceled or when did the customer checked-out of the hotel

**ReservedRoomType:** Code of room type reserved

**StaysInWeekendNights:** Number of weekend nights (Saturday or Sunday) the guest stayed or booked to stay at the hotel

**StaysInWeekNights:** Number of week nights (Monday to Friday) the guest stayed or booked to stay at the hotel

**TotalOfSpecialRequests:** Number of special requests made by the customer (e.g. twin bed or high floor)

### Feature Engineering

The traveling time is represented by both year, date, month, and week. I think only week and year is sufficient.



In [9]:
# drop arrival date month
hotel_1 = hotel_1.drop(['ArrivalDateMonth'],axis=1)
hotel_2 = hotel_2.drop(['ArrivalDateMonth'],axis=1)

# drop arrival date day of month
hotel_1 = hotel_1.drop(['ArrivalDateDayOfMonth'],axis=1)
hotel_2 = hotel_2.drop(['ArrivalDateDayOfMonth'],axis=1)

In [10]:
for col in hotel_1.columns:
    print(col + " : " + str(len(hotel_1[col].unique())) + " features ")
    print("Examples: " + str(hotel_1[col].unique() if len(hotel_1[col].unique()) < 5 else hotel_1[col].unique()[:5]) + "\n")

IsCanceled : 2 features 
Examples: [0 1]

LeadTime : 412 features 
Examples: [342 737   7  13  14]

ArrivalDateYear : 3 features 
Examples: [2015 2016 2017]

ArrivalDateWeekNumber : 53 features 
Examples: [27 28 29 30 31]

StaysInWeekendNights : 16 features 
Examples: [0 1 2 4 3]

StaysInWeekNights : 31 features 
Examples: [0 1 2 3 4]

Adults : 14 features 
Examples: [ 2  1  3  4 40]

Children : 5 features 
Examples: [ 0  1  2 10  3]

Babies : 3 features 
Examples: [0 1 2]

Meal : 5 features 
Examples: ['BB       ' 'FB       ' 'HB       ' 'SC       ' 'Undefined']

Country : 125 features 
Examples: ['PRT' 'GBR' 'USA' 'ESP' 'IRL']

MarketSegment : 6 features 
Examples: ['Direct' 'Corporate' 'Online TA' 'Offline TA/TO' 'Complementary']

DistributionChannel : 4 features 
Examples: ['Direct' 'Corporate' 'TA/TO' 'Undefined']

IsRepeatedGuest : 2 features 
Examples: [0 1]

PreviousCancellations : 11 features 
Examples: [ 0  1  2  3 26]

PreviousBookingsNotCanceled : 31 features 
Examples: [0 